# LAVD and RCLVs on ICON data

Created on: 13.09.2024

Last edited: 13.09.2024

Created by: Stella Bērziņa (stella.berzina@usys.eth.ch)

This notebook aims to document how to calculate LAVD fields from taking input velocity fields from ICON EERIE runs. After LAVD fields are calculated, RCLVs are identified in the fields. Tracking of the RCLVs is also shown. 

This notebook is heavily based on Lexi Jones-Kellett (MIT) example_usage.ipynb which is published along with her code. The functions in this notebook are a combination of **RCLVatlas** by Lexi Jones-Kellett (MIT) (https://github.com/lexi-jones/RCLVatlas) and **xlcs** by Philippe Miron (https://github.com/philippemiron/xlcs). Both initially use **Parcels**  (https://github.com/OceanParcels/Parcels) to create the Lagrangian trajectories and the **Floater** (https://github.com/ocean-transport/floater) to identify the RCLVs. 

My job here is to take the bits that I like from other people's work and alter it to work with the model output data that I have (which I also did not create). So thanks to about a million people for working for years so that I could do this.

## Python Environment
**DOCUMENT LATER**
I would follow the installation steps noted down in RCLVatlas (https://github.com/lexi-jones/RCLVatlas) to have an environment where Parcels runs. 
Also, to make the intake library run, there are a couple of packages

## LAVD calculation

**Input data:**  ICON, 0.25 regridded velocity fields from the intake catalog.

**Output data:** lagrangian trajectories folder, LAVD field.

The run_parcels_CMEMC.py file is changed from the original by feeding in EERIE intake catalogue. 